In [1]:
###1 Extracting all the names of each positive triple from dataset

import pandas as pd
import numpy as np
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

unified_columns = [f"{i}" for i in range(604)]

df = pd.read_csv(os.path.join(parent_dir, 'Step 1 Data Processing/ROBOKOP+DrugMechDB/ROBOKOP+DrugmechDB Data/ROBOMechDB Processed Triples.csv'))
df = df.drop(['Unnamed: 0'],axis=1)
positive_triples_array = []

triples_drug_keys = df['drug_name']
triples_disease_keys = df['disease_name']
triples_protein_keys = df['protein_name']

unique_triples_drug = sorted(list(set(df['drug_name'])))
unique_triples_disease =sorted(list(set(df['disease_name'])))
unique_triples_protein = sorted(list(set(df['protein_name'])))

triples_dictionary = {}
values = my_list = [1] * len(triples_drug_keys)
keys = [triples_drug_keys[i] + " " + triples_disease_keys[i]+ " " + triples_protein_keys[i] for i in range(0,len(triples_drug_keys))]
for key,value in zip(keys,values):
    triples_dictionary[key] = value

for i in range(len(triples_protein_keys)):
    triple = [triples_drug_keys[i],triples_disease_keys[i],triples_protein_keys[i]]
    positive_triples_array.append(triple)

positive_triples_array = np.array(positive_triples_array)

/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24


In [2]:
####2 Importing created protein, disease, and drug vector dictionaries into here

def csvtodict(filename):
    vector_dict = {}
    df_vector_dict = pd.read_csv(filename, header = 0)
    keys = df_vector_dict.iloc[:,0].tolist()
    values = list(range(len(df_vector_dict)))
    for key,value in zip(keys,values):
        vector_dict[key] = value
    return vector_dict

protein_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'))

disease_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'))

drug_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'))

protein_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'),header = 0)
disease_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'),header = 0)
drug_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'), header = 0)

In [3]:
print(positive_triples_array)

[['imatinib' 'chronic myelogenous leukemia, bcr-abl1 positive'
  'abl proto-oncogene 1, non-receptor tyrosine kinase']
 ['imatinib' 'systemic mastocytosis'
  'kit proto-oncogene, receptor tyrosine kinase']
 ['imatinib' 'systemic mastocytosis'
  'platelet derived growth factor receptor alpha']
 ...
 ['theophylline'
  'asthma-chronic obstructive pulmonary disease overlap syndrome'
  'phosphodiesterase 4a']
 ['theophylline'
  'asthma-chronic obstructive pulmonary disease overlap syndrome'
  'phosphodiesterase 4d']
 ['theophylline'
  'asthma-chronic obstructive pulmonary disease overlap syndrome'
  'phosphodiesterase 4b']]


In [4]:
print(len(positive_triples_array))

9478


In [5]:
print(positive_triples_array[1])

['imatinib' 'systemic mastocytosis'
 'kit proto-oncogene, receptor tyrosine kinase']


In [6]:
###3 Building a 600 column vector array of all positive triples

positive_triples_vector_array = []
import itertools

for i in range(0,len(positive_triples_array)):
    try:
        drug_vector = drug_df.iloc[drug_dict[positive_triples_array[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[positive_triples_array[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[positive_triples_array[i,2]],1:201].tolist()
        
        drug_name = positive_triples_array[i,0]
       # drug_id = df.iloc[i,1]
        
        disease_name =positive_triples_array[i,1]
       # disease_id = df.iloc[i,3]
        protein_name = positive_triples_array[i,2]
       # protein_id = df.iloc[i,5]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
        merged = list(itertools.chain(*row))
        positive_triples_vector_array.append(merged)
    except KeyError:
        continue

positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)

print(positive_triples_dataframe)

               0                                                  1    \
0         imatinib    chronic myelogenous leukemia, bcr-abl1 positive   
1         imatinib                              systemic mastocytosis   
2         imatinib                              systemic mastocytosis   
3      paracetamol                                               pain   
4      paracetamol                                               pain   
...            ...                                                ...   
9177   pralidoxime          poisoning of animals with phosphate salts   
9178  theophylline  asthma-chronic obstructive pulmonary disease o...   
9179  theophylline  asthma-chronic obstructive pulmonary disease o...   
9180  theophylline  asthma-chronic obstructive pulmonary disease o...   
9181  theophylline  asthma-chronic obstructive pulmonary disease o...   

                                                    2        3         4    \
0     abl proto-oncogene 1, non-receptor tyro

In [6]:
###4 Use random combinations of drug, disease, protein present in positive triples to form negatives. Create negative vector array
import random

negative_triples_array = []
negative_triples_vector_array = []
temp = set()
i=0
random.seed(42)

while i < 1.05*len(positive_triples_array):  ###create negative triples
    drug = random.sample(unique_triples_drug,k=1)[0]
    protein = random.sample(unique_triples_protein, k=1)[0]
    disease = random.sample(unique_triples_disease,k=1)[0]
    if (drug + " " + disease + " " + protein) in triples_dictionary or (drug + " " + disease + " " + protein) in temp:
        continue
    temp.add(drug + " " + disease + " " + protein)
    negative_triples_array.append([drug, disease, protein])
    i+= 1 

negative_triples_array = np.array(negative_triples_array)
    
for i in range(0,len(negative_triples_array)):
    try:
        drug_vector = drug_df.iloc[drug_dict[negative_triples_array[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[negative_triples_array[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[negative_triples_array[i,2]],1:201].tolist()
        
        drug_name = negative_triples_array[i,0]
        disease_name = negative_triples_array[i,1]
        protein_name = negative_triples_array[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
        merged = list(itertools.chain(*row))
        negative_triples_vector_array.append(merged)

    except KeyError:
        continue

negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)
df1 = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)

df1.columns = unified_columns

#df1.to_csv(os.path.join(os.getcwd(),'Complete Sets/Complete Set 1.csv'))
####--------------------####
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve

shuffled_df = df1.sample(frac=1.0, random_state=42) 
data = shuffled_df.iloc[:, 3:-1].values
labels = shuffled_df.iloc[:, -1].values
print(shuffled_df)
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

all_predictions = []
all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model
fold = 1
auc_values = []

tf.keras.utils.set_random_seed(42)

for train_index, test_index in skf.split(data, labels):
    print(f"Fold: {fold}")
    
    # Split the data into training and test sets
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
    tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
    BATCH_SIZE = 32
    STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
    tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
    tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001,
        decay_steps=STEPS_PER_EPOCH * 1000,
        decay_rate=1,
        staircase=False
    )
        
    def get_optimizer():
        return tf.keras.optimizers.Adam(lr_schedule)

    optimizer = get_optimizer()
        
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(600,)),
        tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
        tf.keras.layers.Dropout(0.8),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.8),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.8),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    # Implement early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
    # Train the model
    model.fit(tf_train_set,
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=40,
                    validation_data = tf_test_set, 
                    callbacks=[early_stopping], 
                    verbose=0)
        
    predictions = model.predict(X_test)
    fpr,tpr, thresholds = roc_curve(y_test,predictions)
    roc_auc = auc(fpr, tpr)
    auc_values.append(roc_auc)
    print("AUROC for test data:", roc_auc)
    all_predictions.append(predictions)
    all_test_sets.append(y_test)
        
    ####Before each iteration ends, save each model###
        
    save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Fold {fold}.keras'))

    fold += 1
        

fcv_mean_auc = np.mean(auc_values)
fcv_std_auc = np.std(auc_values)
fcv_min_auc = np.min(auc_values)
fcv_max_auc = np.max(auc_values)


                           0                                      1  \
4097            theophylline                             bronchitis   
8528              nintedanib              interstitial lung disease   
7621    ethynodiol diacetate              polycystic ovary syndrome   
4754              salbutamol  chronic obstructive pulmonary disease   
15905          prednicarbate                 blepharoconjunctivitis   
...                      ...                                    ...   
11284          isocarboxazid                       male infertility   
11964  loteprednol etabonate     thyroid gland follicular carcinoma   
5390               sunitinib                   renal cell carcinoma   
860              metaraminol                   hypotensive disorder   
15795              icatibant               meningococcal meningitis   

                                                    2         3         4  \
4097                            adenosine a1 receptor  0.556480  0.051

In [ ]:
####5. Generate tpr, tnr, conduct train:test ratio experiment

all_predictions_array = np.round(np.concatenate(all_predictions))
all_test_sets_array = np.concatenate(all_test_sets)

tp_count = 0
fp_count = 0
fn_count = 0
tn_count = 0

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i]) ==1):
        tp_count = tp_count+1

for i in range(0,len(all_test_sets_array)):
     if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==1):
        fp_count =fp_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i])==0):
        fn_count = fn_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==0):
        tn_count = tn_count+1

tpr = tp_count/(tp_count+fn_count)
tnr = tn_count/(tn_count+fp_count)

print('tpr:' , tpr)
print('tnr:' , tnr)
print(fcv_min_auc,fcv_max_auc,fcv_std_auc,fcv_mean_auc)

#####Varying Train:test ratio###

import tensorflow as tf
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, recall_score
import numpy as np
import itertools

stat_array = []
train_var = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.005]
random_states = [42,21,32,10,12]

for i in train_var:
    print('training:testing:', i*100,(1-i)*100)
    auc_values = []
    precision_values = []
    recall_values = []
    for j in range(5):
        tf.keras.utils.set_random_seed(j)
        shuffled_df = df1.sample(frac=1.0, random_state=random_states[j])
        data = np.array(shuffled_df.iloc[:,3:-1])
        labels = np.array(shuffled_df.iloc[:,-1])
        shuffled_df = []

        train_ratio = i
        train_size = int(train_ratio * len(data))
        train_data = data[:train_size]
        test_data = data[train_size:]
        train_labels = labels[:train_size]
        test_labels = labels[train_size:]
        tf_train_set = tf.data.Dataset.from_tensor_slices((train_data,train_labels))
        tf_test_set = tf.data.Dataset.from_tensor_slices((test_data,test_labels))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(train_data)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Train the model
        model.fit(tf_train_set,
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=40,
                    validation_data = tf_test_set, 
                    callbacks=[early_stopping], 
                    verbose=0)
        
        predictions = model.predict(test_data)
        test_predictions_rounded = np.round(predictions)
        fpr, tpr, thresholds = roc_curve(test_labels, predictions)
        precision,recall,thresholds = precision_recall_curve(test_labels, predictions)
        auc_precision_recall = auc(recall, precision)
        recall_score_pred = recall_score(test_labels, test_predictions_rounded)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        precision_values.append(auc_precision_recall)
        recall_values.append(recall_score_pred)
    
    print('auc avg:', np.mean(auc_values), 'auc stddev:', np.std(auc_values))
    print('auprc avg:', np.mean(precision_values), 'auprc stddev:', np.std(precision_values))
    print('recall avg:', np.mean(recall_values), 'recall stddev:', np.std(recall_values))
    stat_array.append([np.mean(auc_values),np.mean(precision_values),np.mean(recall_values)])
    
    #### Exporting AUROC, AUPRC, and Recall values for each train test split as csv
import csv
statfile = "stat.csv"

with open(statfile, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Writing the data row by row
    for row in stat_array:
        csvwriter.writerow(row)


In [ ]:
import csv
statfile = "stat.csv"

with open(statfile, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Writing the data row by row
    for row in stat_array:
        csvwriter.writerow(row)

In [11]:
####5. Complete Set 2 Development and Model Training
import keras
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve

unique_protein_vectors = np.array(protein_df.iloc[:,1:201])
unique_disease_vectors = np.array(disease_df.iloc[:,1:201])
unique_drug_vectors = np.array(drug_df.iloc[:,1:201])

unique_protein_dict_names = np.array(protein_df.iloc[:,0])
unique_disease_dict_names = np.array(disease_df.iloc[:,0])
unique_drug_dict_names = np.array(drug_df.iloc[:,0])


def compute_threshold(vector_dict,n=0.5):
    distances = []
    for i in range((len(vector_dict)-1)):
        smallest_distance = 10000000
        for j in range(i+1, len(vector_dict)):
            distance = np.linalg.norm(vector_dict[i]-vector_dict[j])
            if distance < smallest_distance:
                smallest_distance = distance
        distances.append(smallest_distance)

    distances = np.array(distances)
    print(len(distances))
    print('min', np.min(distances))
    mean_distances = np.sum(distances)/len(distances)
    stdev_distance = np.std(distances)
    print('mean',mean_distances)
    print('stdev',stdev_distance)

    applicability_domain = mean_distances+(n*stdev_distance)

    return applicability_domain

drug_threshold = compute_threshold(unique_drug_vectors)
disease_threshold = compute_threshold(unique_disease_vectors)
protein_threshold = compute_threshold(unique_protein_vectors)

print(drug_threshold)
print(disease_threshold)
print(protein_threshold)

random_rows = positive_triples_dataframe.sample(int(len(positive_triples_dataframe)/4), random_state = 42)

#we are now going to split these rows into 3 equal chunks. 
#the triples in the first chunk will have its protein parameter randomized, 
#the triples in the second chunk will have its disease parameter randomized, and so on

split_rows = np.array_split(random_rows, 3)

# Each part is now a separate DataFrame
drug_disease_x, drug_x_protein, x_disease_protein = split_rows[0], split_rows[1], split_rows[2]

drug_disease_x_negative_triples = []
drug_x_protein_negative_triples = []
x_disease_protein_negative_triples = []
temp = set()

for i in range(int(len(drug_disease_x)/1.1)):
    reference = np.array(drug_disease_x.iloc[i, 403:603]) ###not including 401, including 601
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_protein_vectors)):
        if (np.linalg.norm(reference-unique_protein_vectors[j]) < protein_threshold):
            triple_name = [drug_disease_x.iloc[i,0], drug_disease_x.iloc[i,1], unique_protein_dict_names[j]]
            new_combo = drug_disease_x.iloc[i,0] + " " + drug_disease_x.iloc[i,1] + " " + unique_protein_dict_names[j]
            if (new_combo in triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_disease_x.iloc[i,3:403]).tolist(), unique_protein_vectors[j].tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_disease_x_negative_triples.append(row)
            counter+=1
            if counter >=6:
                break

for i in range(len(drug_x_protein)):
    reference = np.array(drug_x_protein.iloc[i, 203:403])
    reference = [float(item) for item in reference]
    counter=0
    for j in range(len(unique_disease_vectors)):
        if (np.linalg.norm(reference-unique_disease_vectors[j]) < disease_threshold):
            triple_name = [drug_x_protein.iloc[i,0], unique_disease_dict_names[j],drug_x_protein.iloc[i,2]]
            new_combo = drug_x_protein.iloc[i,0] + " " + unique_disease_dict_names[j] + " " + drug_x_protein.iloc[i,2]
            if (new_combo in triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[np.array(drug_x_protein.iloc[i,3:203]).tolist(), unique_disease_vectors[j].tolist(),np.array(drug_x_protein.iloc[i,403:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            drug_x_protein_negative_triples.append(row)
            counter +=1
            if counter >=6:
                break

for i in range(len(x_disease_protein)):
    reference = np.array(x_disease_protein.iloc[i, 3:203])
    reference = [float(item) for item in reference]
    counter=0    
    for j in range(len(unique_drug_vectors)):
        if (np.linalg.norm(reference-unique_drug_vectors[j]) < drug_threshold):
            triple_name = [unique_drug_dict_names[j],x_disease_protein.iloc[i,1], x_disease_protein.iloc[i,2]]
            new_combo = unique_drug_dict_names[j] + " " + x_disease_protein.iloc[i,1] + " " + x_disease_protein.iloc[i,2]
            if (new_combo in triples_dictionary or new_combo in temp):
                continue
            temp.add(new_combo)
            triple_vector = list(itertools.chain(*[unique_drug_vectors[j].tolist(),np.array(x_disease_protein.iloc[i,203:603]).tolist()]))
            row = list(itertools.chain(*[triple_name,triple_vector,[0]]))
            x_disease_protein_negative_triples.append(row)
            counter += 1
            if counter >=5:
                break
            
negative_df = pd.concat([pd.DataFrame(drug_disease_x_negative_triples),pd.DataFrame(drug_x_protein_negative_triples),pd.DataFrame(x_disease_protein_negative_triples)],axis=0,ignore_index=True)
df_2 = pd.concat([positive_triples_dataframe,negative_df],axis=0)
df_2.columns = unified_columns
df_2.to_csv(os.path.join(os.getcwd(),'Complete Sets/Complete Set 2.csv'), index=False)

shuffled_df = df_2.sample(frac=1.0, random_state=i)

data = shuffled_df.iloc[:,3:-1].values
labels = shuffled_df.iloc[:,-1].values

num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

all_predictions = []
all_test_sets = []

# Perform 5-fold cross-validation to test the accuracy of the model######
fold = 1
auc_values = []

keras.utils.set_random_seed(i)
#tf.config.experimental.enable_op_determinism()

shuffled_df = df_2.sample(frac=1.0, random_state=i)

data = shuffled_df.iloc[:,3:-1].values
labels = shuffled_df.iloc[:,-1].values

num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

all_predictions = []
all_test_sets = []

# Perform 5-fold cross-validation to test the accuracy of the model######
fold = 1
auc_values = []

keras.utils.set_random_seed(i)
    #tf.config.experimental.enable_op_determinism()

for train_index, test_index in skf.split(data, labels):
    print(f"Fold: {fold}")
    print(train_index)
    
    # Split the data into training and test sets
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
    tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
    BATCH_SIZE = 32
    STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
    tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
    tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001,
        decay_steps=STEPS_PER_EPOCH * 1000,
        decay_rate=1,
        staircase=False
    )
        
    def get_optimizer():
        return tf.keras.optimizers.Adam(lr_schedule)

    optimizer = get_optimizer()
        
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(600,)),
        tf.keras.layers.Dense(1024, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)), tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    # Implement early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    # Train the model2
    model.fit(tf_train_set,
                steps_per_epoch=STEPS_PER_EPOCH, 
                epochs=100,
                validation_data = tf_test_set, 
                callbacks=[early_stopping], 
                verbose=0)
        
    predictions = model.predict(X_test)
    fpr,tpr, thresholds = roc_curve(y_test,predictions)
    roc_auc = auc(fpr, tpr)
    auc_values.append(roc_auc)
    print("AUROC for test data:", roc_auc)
    all_predictions.append(predictions)
    all_test_sets.append(y_test)
        
    ####Before each iteration ends, save each model###
        
    save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 2 Models/ROBOMechDB Complete Set 2 Model Fold {fold}.keras'))

    fold += 1

fcv_mean_auc = np.mean(auc_values)
fcv_std_auc = np.std(auc_values)
fcv_min_auc = np.min(auc_values)
fcv_max_auc = np.max(auc_values)
print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)

all_predictions_array = np.round(np.concatenate(all_predictions))
all_test_sets_array = np.concatenate(all_test_sets)

tp_count = 0
fp_count = 0
fn_count = 0
tn_count = 0

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i]) ==1):
        tp_count = tp_count+1

for i in range(0,len(all_test_sets_array)):
     if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==1):
        fp_count =fp_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i])==0):
        fn_count = fn_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==0):
        tn_count = tn_count+1

tpr = tp_count/(tp_count+fn_count)
tnr = tn_count/(tn_count+fp_count)

print('tpr:' , tpr)
print('tnr:' , tnr)
print(fcv_min_auc,fcv_max_auc,fcv_std_auc,fcv_mean_auc)

all_predictions_array = np.round(np.concatenate(all_predictions))
all_test_sets_array = np.concatenate(all_test_sets)

tp_count = 0
fp_count = 0
fn_count = 0
tn_count = 0

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i]) ==1):
        tp_count = tp_count+1

for i in range(0,len(all_test_sets_array)):
     if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==1):
        fp_count =fp_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==1 and int(all_predictions_array[i])==0):
        fn_count = fn_count+1

for i in range(0,len(all_test_sets_array)):
    if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==0):
        tn_count = tn_count+1

tpr = tp_count/(tp_count+fn_count)
tnr = tn_count/(tn_count+fp_count)

print('tpr:' , tpr)
print('tnr:' , tnr)
print(fcv_min_auc,fcv_max_auc,fcv_std_auc,fcv_mean_auc)


998
min 1.3490766481180738
mean 3.9296513978117704
stdev 1.1837539597778333
589
min 1.3756730714527872
mean 6.979102681442049
stdev 2.672071221220308
835
min 1.00340273053115
mean 6.690955269076238
stdev 4.105268395143029
4.521528377700687
8.315138292052202
8.743589466647753


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Fold: 1
[    1     2     3 ... 17918 17919 17921]
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
AUROC for test data: 0.7798462260002567
Fold: 2
[    0     1     2 ... 17919 17920 17921]
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
AUROC for test data: 0.7510099418388104
Fold: 3
[    0     1     3 ... 17919 17920 17921]
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step
AUROC for test data: 0.7648760114266913
Fold: 4
[    0     2     3 ... 17919 17920 17921]
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step
AUROC for test data: 0.747308938195161
Fold: 5
[    0     1     2 ... 17916 17918 17920]
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
AUROC for test data: 0.7723487596157201
0.7630779754153278 0.747308938195161 0.7798462260002567
tpr: 0.815726421258985
tnr: 0.5675057208237986
0.747308938195161 0.7798462260002567 0.012366523926490649 0.7630779754153278
tpr: 0.815726421258985
tnr: 0.5675057208237986
0.747308938195161 0.7798462260002567 0.012366523926490649 0.7630779754153278


/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_93328/3996162156.py:240: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if(all_test_sets_array[i]==1 and int(all_predictions_array[i]) ==1):
/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_93328/3996162156.py:244: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if(all_test_sets_array[i]==0 and int(all_predictions_array[i])==1):
/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_93328/3996162156.py:248: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array bef

In [ ]:
###6. #Varying Train:test ratio###

import tensorflow as tf
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, recall_score
import numpy as np
import itertools

stat_array = []
train_var = [0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.005]
random_states = [42,21,32,10,12]

for i in train_var:
    print('training:testing:', i*100,(1-i)*100)
    auc_values = []
    precision_values = []
    recall_values = []
    for j in range(5):
        tf.keras.utils.set_random_seed(j)
        shuffled_df = df_2.sample(frac=1.0, random_state=random_states[j])
        data = np.array(shuffled_df.iloc[:,3:-1])
        labels = np.array(shuffled_df.iloc[:,-1])
        shuffled_df = []

        train_ratio = i
        train_size = int(train_ratio * len(data))
        train_data = data[:train_size]
        test_data = data[train_size:]
        train_labels = labels[:train_size]
        test_labels = labels[train_size:]
        tf_train_set = tf.data.Dataset.from_tensor_slices((train_data,train_labels))
        tf_test_set = tf.data.Dataset.from_tensor_slices((test_data,test_labels))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

        # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)), tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        # Train the model
        model.fit(tf_train_set,
                steps_per_epoch=STEPS_PER_EPOCH, 
                epochs=100,
                validation_data = tf_test_set, 
                callbacks=[early_stopping], 
                verbose=0)
        
        predictions = model.predict(test_data)
        test_predictions_rounded = np.round(predictions)
        fpr, tpr, thresholds = roc_curve(test_labels, predictions)
        precision,recall,thresholds = precision_recall_curve(test_labels, predictions)
        auc_precision_recall = auc(recall, precision)
        recall_score_pred = recall_score(test_labels, test_predictions_rounded)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        precision_values.append(auc_precision_recall)
        recall_values.append(recall_score_pred)
    
    print('auc avg:', np.mean(auc_values), 'auc stddev:', np.std(auc_values))
    print('auprc avg:', np.mean(precision_values), 'auprc stddev:', np.std(precision_values))
    print('recall avg:', np.mean(recall_values), 'recall stddev:', np.std(recall_values))
    stat_array.append([np.mean(auc_values),np.mean(precision_values),np.mean(recall_values)])
    
    #### Exporting AUROC, AUPRC, and Recall values for each train test split as csv
import csv
statfile = "stat.csv"

with open(statfile, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Writing the data row by row
    for row in stat_array:
        csvwriter.writerow(row)


In [ ]:

import csv

statfile = "stat.csv"

with open(statfile, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Writing the data row by row
    for row in stat_array:
        csvwriter.writerow(row)